#ANALISIS TEMATIK

In [ ]:
!pip install geemap
import geemap
import ee

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00


In [ ]:
# import ee
# ee.Authenticate()
# ee.Initialize()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-kelompok4geospasial')

In [ ]:
# #Tentukan titik koordinat (contoh: Jakarta)
# latitude = -6.2086
# longitude = 106.8456

# #Buat objek geometri titik
# point = ee.Geometry.Point(longitude, latitude)

# #Buat lingkaran (buffer) dengan jari-jari 5 km]
# buffered_point = point.buffer(5000)

# #Inisialisasi peta interaktif
# Map = geemap.Map()
# Map.centerObject(point, 10)
# Map.addLayer(buffered_point, {'color':'FF0000'}, 'Radius 5 Km')
# Map.addLayer(point)
# Map

#Radius dan Beberapa Titik Lokasi

In [ ]:
!pip install folium

In [ ]:
import folium
import pandas as pd

In [ ]:
#FUngsi untuk menambahkan marker pada peta folium
def add_marker(map_obj, location, popup_text):
  marker = folium.Marker(location=location, popup=popup_text)
  marker.add_to(map_obj)

#Baca data dari File CSV
file_path= '#13 Dataset Praktik.xlsx'
data = pd.read_excel(file_path)

In [ ]:
latitude = data['Latitude'].mean()
longitude = data['Longitude'].mean()

In [ ]:
#Buat peta folium
map_obj = folium.Map(location=[latitude, longitude], zoom_start = 13)

#FUngsi untuk menambhakan marker ke pusat centroid pada peta folium
def add_marker_red(map_obj, location, popup_text, icon_color = 'red'):
  marker = folium.Marker(location=location, popup=popup_text, icon = folium.Icon(color=icon_color))
  marker.add_to(map_obj)

#Tambahkan marker lokasi pusat (centroid Jakarta)
add_marker_red(map_obj, (latitude,longitude ), 'LOKASI CENTROID')

#Tambahkan marker untuk setiap lokasi dari data csv
Dataset_baru=[]
for index, row in data.iterrows():
  location = [row['Latitude'], row['Longitude']]
  popup_text = f"Label: {row['Lokasi']}"
  add_marker(map_obj, location, popup_text)

In [ ]:
#Membuat Radius
point = ee.Geometry.Point(latitude, longitude)

#Buat lingkaran (buffer) dengan jari-jari 3 km]
buffered_point = point.buffer(3000)

#Set lokasi pusat peta pada titik
map_obj.location = [latitude, longitude]

#Tambahkan lingkaran ke peta
folium.Circle(location=[latitude, longitude],
              radius = 3000,
              color = 'FF0000',
              fill = True,
              fil_color = 'FF0000').add_to(map_obj)

map_obj

#Data dalam Radius

In [ ]:
#FUngsi menghitung jarak dengan harvesine
import math

def harvesine_distance(lat1, lon1, lat2, lon2):
  #Radius Bumi dalam KM
  R = 6371.0

  #Konversi lintang dan bujur daru derajat ke radian
  lat1_rad = math.radians(lat1)
  lon1_rad = math.radians(lon1)
  lat2_rad = math.radians(lat2)
  lon2_rad = math.radians(lon2)

  #perbedaan koordinat
  dlat = lat2_rad - lat1_rad
  dlon = lon2_rad - lon1_rad

  #RUmus Haversine
  a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

  #Hitung jarak
  distance = R * c

  return distance

In [ ]:
#data dalam Radius
data_rad = []
lat1 = latitude
lon1 = longitude
for index, row in data.iterrows():
  location = [row['Latitude'], row['Longitude']]
  jarak_km = harvesine_distance(lat1, lon1, row['Latitude'], row['Longitude'])
  nama_lokasi = f"{row['Lokasi']}"
  biaya = f"{row['Biaya']}"
  rating = f"{row['Rating']}"
  if jarak_km <= 3:
    data_rad.append({'Koordinat': location, 'Nama Lokasi' : nama_lokasi, 'Biaya': biaya, 'Jarak' : jarak_km, 'Rating':rating })

#Tampilan data_rad
data_rad = pd.DataFrame(data_rad)
data_rad

,Koordinat,Nama Lokasi,Biaya,Jarak,Rating
0,"[-6.221339, 106.868167]",A,132598,2.444792,3
1,"[-6.206947, 106.842225]",B,64402,0.971642,4
2,"[-6.210023, 106.823916]",D,128139,2.622743,4
3,"[-6.232302, 106.861342]",F,134851,2.552341,5
4,"[-6.218691, 106.85145]",H,130006,0.688071,2
5,"[-6.213202, 106.834877]",I,64960,1.375344,4
6,"[-6.20958, 106.82305]",K,50007,2.725722,5
7,"[-6.20998, 106.842835]",N,66462,0.671648,4
8,"[-6.239924, 106.84518]",P,83098,2.881935,5
9,"[-6.214801, 106.83069]",Q,149623,1.836322,5


#Rekomendasi Lokasi Target

In [ ]:
#data dalam radius
df= pd.DataFrame(data_rad)

#cek nilai Biaya dan Rating
if 'Biaya' in df.columns and 'Rating' in df.columns:
  #convert biaya dan rating ke numerik
  df['Biaya'] = pd.to_numeric(df['Biaya'], errors='coerce')
  df['Jarak'] = pd.to_numeric(df['Jarak'], errors='coerce')
  df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

  #cek kesalahan konversi
  if df[['Biaya', 'Jarak', 'Rating']].isnull().values.any():
    print("Warning: Kolom Biaya, Jarak dan Rating bukan numerik")

  #normalization
  df['Normalized Biaya'] = df['Biaya'].min()/df['Biaya']
  df['Normalized Jarak'] = df['Jarak'].min()/df['Jarak']
  df['Normalized Rating'] = df['Rating']/df['Rating'].max()

  #Weights/Pembobotann areibut biaya dan rating
  bobot_biaya = 0.5
  bobot_jarak = 0.2
  bobot_rating = 0.3

  #Perhitungan total bobot prioritas (SUm of Product)
  df['Nilai Terbobot'] = bobot_biaya * df['Normalized Biaya'] +  bobot_jarak * df['Normalized Jarak'] + bobot_rating * df['Normalized Rating']

  #pilih data terbaik
  data_terbaik = df[df['Nilai Terbobot'] == df['Nilai Terbobot'].max()]
  print("Rekomendasi Terbaik:")
  print(data_terbaik)

else:
  print("BIaya atau Rating tidak ditemukan")

Rekomendasi Terbaik:
               Koordinat Nama Lokasi  Biaya     Jarak  Rating  \
6  [-6.20958, 106.82305]           K  50007  2.725722       5   

   Normalized Biaya  Normalized Jarak  Normalized Rating  Nilai Terbobot  
6               1.0          0.178283                1.0        0.835657  


In [ ]:
#Pemberian marker warna hijau pada lokasi target
def add_marker_green(map_obj, location, popup_text):
  marker = folium.Marker(location=location, popup = popup_text, icon=folium.Icon(color='green'))
  marker.add_to(map_obj)

#Tambahkan marker lokasi pusat
location = (latitude, longitude)
add_marker_green(map_obj, location, 'LOKASI TARGET')
map_obj

In [ ]:
#Tampilkan Target Lokasi Tujuan
best_lat = data_terbaik['Koordinat'].iloc[0][0]
best_lon = data_terbaik['Koordinat'].iloc[0][1]

jarak_hasil = harvesine_distance(latitude, longitude, best_lat, best_lon)
print("Jarak Lokasi Centroid Dengan Lokasi Target = ", jarak_hasil)

Jarak Lokasi Centroid Dengan Lokasi Target =  2.7257224406875205
